In [1]:
!pip install importnb 
!pip install ipywidgets 
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager


In [2]:
from importnb import Notebook
with __import__('importnb').Notebook():
    from scripts import *

In [3]:
%%capture
from tqdm.notebook import tqdm as tqdm

In [4]:
import json
import os
import sys
import pprint
import traceback
from datetime import datetime
import requests
import time

In [5]:
def get_current_job_id(flink_address):
    print("Running get_current_job_id in " + flink_address)
    out = subprocess.check_output("set -e pipefail && curl -s {flink_address}/jobs/ | jq '.jobs[].id' | tr -d '\"'".format(flink_address=flink_address),
                                   shell=True)
    print(out)
    print("Finished get_current_job_id is %s" % out.decode("UTF-8").strip())

    return out.decode("UTF-8").strip()

def rescale(number):
    print("Clean logs and old job artifacts")
    run_command("kubectl scale deployment -n default flink-jobmanager --replicas=0", shell=False)
    run_command("kubectl scale deployment -n default flink-taskmanager --replicas=0", shell=False)
    time.sleep(5)
    run_command("kubectl scale deployment -n default flink-jobmanager --replicas=1", shell=False)
    run_command("kubectl scale deployment -n default flink-taskmanager --replicas={}".format(number), shell=False)
    time.sleep(60)
    return 


In [6]:
def install_chart(path, name, params, timeout="180s"):
    with cd(path):
        param_str = ""
        for k in params.keys():
            param_str += "--set {}={} ".format(str(k).replace("!","."), str(params[k]).replace("!","."))
        command = "helm install {}  . {} --wait  --namespace default --timeout {} ".format(name, param_str, timeout)
        result = run_command(command, log = shell_log)
        return result   
def uninstall_chart(path, name, timeout="120s"):
    with cd(path):
        result = run_command("helm delete {} --namespace default".format(name), log = shell_log)
        print(result)
        if result == "0":
            sleep(timeout)
    return result    

In [7]:
def reinstall(install_name, version, params=None):
    uninstall_chart("../charts/flink", install_name)
    sleep(60)
    if params is None:
        params = {
            "image!tag":version
        }
    install_chart("../charts/flink", install_name, params)

In [8]:
def install(version, params=None):
    install_name = "flink" 
    if params is None:
        params = dict(default_params_flink)
    params["image!tag"] = version
    reinstall(install_name, version, params)
    
def test(chart_name,bucket=None, default_params=None, specific_params={}, taskmanager_quantity=30, no_rescale=False):
   
    if default_params is None:
        return None
    
    params = default_params.copy()
    for name in specific_params:
        params[name] = specific_params[name]
    print(params)
    params["bucket"] = bucket

    uninstall_chart("../charts/flink-job", chart_name)
    if not no_rescale:
        rescale(taskmanager_quantity)
        sleep(60)
    
    install_chart("../charts/flink-job", chart_name, params)
    ret = run_command("kubectl -n default wait --for=condition=complete --timeout={}s job/{}".format(600, chart_name + "-flink-job-retrieve"))
    
    return ret

In [9]:
shell_log = False
default_params_flink = {
    "taskmanager!replicaCount": 1,
    "hdfs!persistence!nameNode!storageClass": "local-path",
    "hdfs!persistence!dataNode!storageClass": "local-path",
    "hdfs!persistence!nameNode!size": "5Gi",
    "hdfs!persistence!dataNode!size": "5Gi",
    "zookeeper!persistence!storageClass": "local-path",
    "kafka!persistence!storageClass": "local-path",
    "hdfs!enabled":"false"
}

duration = 120
default_params_ar = {
    #"testName":test_name,
    #"bucket":"increasing-length3",
    #"algorithm": algorithm,
    "image!tag": "1.7.2-ar",
    "installKafka": "false",
    "job!jobClass":"me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob",
    "job!length":1,
    "job!duration": duration,
    "job!mapParallelism": "2",
    "algorithm!checkpointing":"false",
    "algorithm!stateBackend":"rocksdb",
    "algorithm!idleMarks": "true",
    "job!sharingGroup":"true",
    "installHdfs":"true",
    "job!bufferTimeout": 5
}  
default_params_vanilla = {
    #"testName":test_name,
    #"bucket":"increasing-length3",
    "image!tag": "1.7.2-baseline",
    "installKafka": "false",
    "algorithm!type": "VANILLA",
    "job!jobClass":"me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob",
    "job!length":1,
    "job!duration": duration,
    "job!mapParallelism": "2",
    "algorithm!checkpointing":"true",
    "algorithm!stateBackend":"rocksdb",
    "algorithm!idleMarks": "false",
    "job!sharingGroup":"true",
    "killTaskManager!waitForKill": "True",
    "installHdfs":"true",
    "job!bufferTimeout": 5    
}

## parameters initialization

In [10]:
# VANILLA NO CHECKPOINTING
params_vanilla_nocp = dict(default_params_vanilla)
params_vanilla_nocp["name_algorithm"] = "nc"
params_vanilla_nocp["algorithm!type"] = "VANILLA_NOCP"
params_vanilla_nocp["algorithm!checkpointing"] = "false"
# VANILLA CHECKPOINTING
params_vanilla = dict(default_params_vanilla)
params_vanilla["name_algorithm"] = "cp"
params_vanilla["algorithm!type"] = "VANILLA"
params_vanilla["installHdfs"] = "true"
# ACTIVE_REPLICATION NO_ORDERING
params_ar_noordering = dict(default_params_ar)
params_ar_noordering["name_algorithm"] = "no"
params_ar_noordering["algorithm!type"] = "NO_ORDERING"
# ACTIVE_REPLICATION BETTER_BIAS
params_ar_betterbias = dict(default_params_ar)
params_ar_betterbias["name_algorithm"] = "tm"
params_ar_betterbias["algorithm!type"] = "BETTER_BIAS"
params_ar_betterbias["algorithm!idleMarksInterval"] = 25
# ACTIVE_REPLICATION BETTER_BIAS 
params_ar_betterbias_nosp = dict(default_params_ar)
params_ar_betterbias_nosp["name_algorithm"] = "tm"
params_ar_betterbias_nosp["algorithm!type"] = "BETTER_BIAS"
params_ar_betterbias_nosp["algorithm!idleMarks"] = "false"

# ACTIVE_REPLICATION LEADER_KAFKA
params_ar_leader_kafka = dict(default_params_ar)
params_ar_betterbias_nosp["algorithm!idleMarks"] = "false"
params_ar_leader_kafka["name_algorithm"] = "ka"
params_ar_leader_kafka["algorithm!type"] = "LEADER_KAFKA"
params_ar_leader_kafka["installKafka"] = "true"

# kill Leader
params_ar_leader_kafka_kill_leader = dict(params_ar_leader_kafka)
params_ar_leader_kafka_kill_leader["killTaskManager!kazooTarget"] = "leader"
params_ar_leader_kafka_kill_follower = dict(params_ar_leader_kafka)
params_ar_leader_kafka_kill_follower["killTaskManager!kazooTarget"] = "follower"

# liverobin
params_ar_liverobin = dict(default_params_ar)
params_ar_liverobin["algorithm!type"] = "LIVE_ROBIN" 
params_ar_liverobin["algorithm!liveRobinMarks"] = "true" 
params_ar_liverobin["algorithm!idleMarks"] = "false" 
params_ar_liverobin["name_algorithm"] = "lr"
params_ar_liverobin["installKafka"] = "true"
params_ar_liverobin["algorithm!heartbeatEmitter!enabled"] = "true"
params_ar_liverobin["algorithm!liveRobinMarksInterval"] = 10

                   

# mb1 v2

In [ ]:
bucket = "mb1-v2-noprom-final"
length = 1
test_maps = [params_vanilla_nocp, params_vanilla, params_ar_noordering, params_ar_liverobin, params_ar_leader_kafka]
#test_maps = [params_ar_leader_kafka]
for test_map in tqdm(test_maps):
    flink_params = dict(default_params_flink)
    flink_params["kafka!enabled"] = test_map["installKafka"]
    flink_params["zookeeper!enabled"] = test_map["installKafka"]
    flink_params["hdfs!enabled"] = test_map["installHdfs"]    
    install(test_map["image!tag"], flink_params)
    for length in tqdm(range(1,5)):    
        for index in tqdm(range(5)):
            test("mb1-{}-{}-{}".format(test_map["name_algorithm"], length, index),
                 bucket,
                 test_map,
                 {
                   "job!length":length
                 },
                 no_rescale=False,
                 taskmanager_quantity=29
                )   
            

# mb2

In [13]:
bucket = "mb2-v2-noprom"
length = 1
test_maps = [params_vanilla_nocp, params_vanilla, params_ar_noordering, params_ar_liverobin, params_ar_leader_kafka]
#test_maps = [params_vanilla_nocp]
for test_map in tqdm(test_maps):
    flink_params = dict(default_params_flink)
    flink_params["kafka!enabled"] = test_map["installKafka"]
    flink_params["zookeeper!enabled"] = test_map["installKafka"]
    flink_params["hdfs!enabled"] = test_map["installHdfs"]    
    install(test_map["image!tag"], flink_params)
    for parallel in tqdm(range(1,5)):    
        for index in tqdm(range(5)):
            test("mb2-{}-{}-{}".format(test_map["name_algorithm"], parallel, index),
                 bucket,
                 test_map,
                 {
                   "job!mapParallelism":parallel
                 },
                 no_rescale=False,
                 taskmanager_quantity=29
                )


helm delete flink --namespace default
0
helm install flink  . --set taskmanager.replicaCount=1 --set hdfs.persistence.nameNode.storageClass=local-path --set hdfs.persistence.dataNode.storageClass=local-path --set hdfs.persistence.nameNode.size=5Gi --set hdfs.persistence.dataNode.size=5Gi --set zookeeper.persistence.storageClass=local-path --set kafka.persistence.storageClass=local-path --set hdfs.enabled=true --set kafka.enabled=false --set zookeeper.enabled=false --set image.tag=1.7.2-baseline  --wait  --namespace default --timeout 180s 


{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA_NOCP', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': 1, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'nc'}
helm delete mb2-nc-1-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-08 20:13:18 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-08 20:13:19 : deployment.apps/flink-taskmanager scaled
kubectl scale deployment -n default flink

{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA_NOCP', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': 2, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'nc'}
helm delete mb2-nc-2-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-08 20:38:10 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-08 20:38:11 : deployment.apps/flink-taskmanager scaled
kubectl scale deployment -n default flink

{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA_NOCP', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': 3, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'nc'}
helm delete mb2-nc-3-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-08 21:03:04 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-08 21:03:04 : deployment.apps/flink-taskmanager scaled
kubectl scale deployment -n default flink

{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA_NOCP', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': 4, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'nc'}
helm delete mb2-nc-4-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-08 21:28:05 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-08 21:28:05 : deployment.apps/flink-taskmanager scaled
kubectl scale deployment -n default flink

...

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': 4, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 10}
helm delete mb2-lr-4-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 02:44:37 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 02:44:37 : deployme

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': 1, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA'}
helm delete mb2-ka-1-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 03:13:45 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 03:13:46 : deployment.apps/flink-taskmanager scaled
kubectl scale deployment -n default flink-jobmanager --replicas=1
kubectl scale deployme

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': 2, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA'}
helm delete mb2-ka-2-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 03:40:10 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 03:40:10 : deployment.apps/flink-taskmanager scaled
kubectl scale deployment -n default flink-jobmanager --replicas=1
kubectl scale deployme

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': 3, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA'}
helm delete mb2-ka-3-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 04:06:32 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 04:06:33 : deployment.apps/flink-taskmanager scaled
kubectl scale deployment -n default flink-jobmanager --replicas=1
kubectl scale deployme

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': 4, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA'}
helm delete mb2-ka-4-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 04:33:10 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 04:33:10 : deployment.apps/flink-taskmanager scaled
kubectl scale deployment -n default flink-jobmanager --replicas=1
kubectl scale deployme

In [ ]:
print([x for x in range(3)])

# mb3

In [13]:
bucket = "mb3-v2-new2"

#test_maps = [params_ar_leader_kafka]
#test_maps = [params_ar_leader_kafka]
test_maps = [params_vanilla, params_ar_liverobin, params_ar_leader_kafka_kill_leader, params_ar_leader_kafka_kill_follower]
#test_maps = [params_vanilla]
#test_maps = [params_vanilla, params_ar_betterbias, params_ar_betterbias_nosp, params_ar_leader_kafka]
for test_map in tqdm(test_maps):
    flink_params = dict(default_params_flink)
    flink_params["kafka!enabled"] = test_map["installKafka"]
    flink_params["zookeeper!enabled"] = test_map["installKafka"]
    flink_params["hdfs!enabled"] = test_map["installHdfs"]    
    
    for task in tqdm(range(1)):
        for index in tqdm(range(1)):
            install(test_map["image!tag"], flink_params)
            test_name = format("{}-{}-{}-{}-{}".format(datetime.today().strftime('%Y-%m-%d-%H-%M-%S'), bucket, test_map["name_algorithm"], task, index))
            test("mb3-{}-{}".format(test_map["name_algorithm"], index),
                 bucket,
                 test_map,
                 {
                    "experimentationName": test_name,
                    "retrieveScript!getAllSinks":"true",
                    "job!mapParallelism":2,
                    "job!length":3,
                    "job!rate":5000,
                    "job!duration": 300,                 
                    "killTaskManager!enabled": "true",
                    "killTaskManager!taskName": "map-at-level-{}".format(task),
                    "killTaskManager!killDelay": 150,
                    "killTaskManager!replicaIndex": 0,
                    "killTaskManager!operatorIndex": 0,
                    "killTaskManager!gracePeriod": 0,
                    "algorithm!heartbeatEmitter!duration": 450
                 },
                 no_rescale=False,
                 taskmanager_quantity=29
                )

helm delete flink --namespace default
0
helm install flink  . --set taskmanager.replicaCount=1 --set hdfs.persistence.nameNode.storageClass=local-path --set hdfs.persistence.dataNode.storageClass=local-path --set hdfs.persistence.nameNode.size=5Gi --set hdfs.persistence.dataNode.size=5Gi --set zookeeper.persistence.storageClass=local-path --set kafka.persistence.storageClass=local-path --set hdfs.enabled=true --set kafka.enabled=false --set zookeeper.enabled=false --set image.tag=1.7.2-baseline  --wait  --namespace default --timeout 180s 
{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 3, 'job!duration': 300, 'job!mapParallelism': 2, 'algorithm!checkpointing': 'true', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algor

helm delete flink --namespace default
0
helm install flink  . --set taskmanager.replicaCount=1 --set hdfs.persistence.nameNode.storageClass=local-path --set hdfs.persistence.dataNode.storageClass=local-path --set hdfs.persistence.nameNode.size=5Gi --set hdfs.persistence.dataNode.size=5Gi --set zookeeper.persistence.storageClass=local-path --set kafka.persistence.storageClass=local-path --set hdfs.enabled=true --set kafka.enabled=true --set zookeeper.enabled=true --set image.tag=1.7.2-ar  --wait  --namespace default --timeout 180s 
{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 3, 'job!duration': 300, 'job!mapParallelism': 2, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', '

helm delete flink --namespace default
0
helm install flink  . --set taskmanager.replicaCount=1 --set hdfs.persistence.nameNode.storageClass=local-path --set hdfs.persistence.dataNode.storageClass=local-path --set hdfs.persistence.nameNode.size=5Gi --set hdfs.persistence.dataNode.size=5Gi --set zookeeper.persistence.storageClass=local-path --set kafka.persistence.storageClass=local-path --set hdfs.enabled=true --set kafka.enabled=true --set zookeeper.enabled=true --set image.tag=1.7.2-ar  --wait  --namespace default --timeout 180s 
{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 3, 'job!duration': 300, 'job!mapParallelism': 2, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA', 'killTaskManager!kazooTarget': 'lead

helm delete flink --namespace default
0
helm install flink  . --set taskmanager.replicaCount=1 --set hdfs.persistence.nameNode.storageClass=local-path --set hdfs.persistence.dataNode.storageClass=local-path --set hdfs.persistence.nameNode.size=5Gi --set hdfs.persistence.dataNode.size=5Gi --set zookeeper.persistence.storageClass=local-path --set kafka.persistence.storageClass=local-path --set hdfs.enabled=true --set kafka.enabled=true --set zookeeper.enabled=true --set image.tag=1.7.2-ar  --wait  --namespace default --timeout 180s 
{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 3, 'job!duration': 300, 'job!mapParallelism': 2, 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA', 'killTaskManager!kazooTarget': 'foll

## mb4

In [14]:
bucket = "mb4-v2-noprom"
length = 1
#test_maps = [params_vanilla, params_ar_betterbias]
#test_maps = [params_ar_leader_kafka]
test_maps = [params_vanilla, params_ar_liverobin, params_ar_leader_kafka]
#test_maps = [params_vanilla]

for test_map in tqdm(test_maps):
    flink_params = dict(default_params_flink)
    flink_params["kafka!enabled"] = test_map["installKafka"]
    flink_params["zookeeper!enabled"] = test_map["installKafka"]
    flink_params["hdfs!enabled"] = test_map["installHdfs"]
    
    install(test_map["image!tag"], flink_params)
    for rate in tqdm([10,100,1000,5000,10000]):
        for index in tqdm(range(5)):
#    for rate in tqdm([1, 10, 100, 1000]):
#        for index in tqdm(range(5)):
            test_name = format("{}-{}-{}-{}".format(datetime.today().strftime('%Y-%m-%d-%H-%M-%S'), bucket, test_map["name_algorithm"], index))
            test("mb4-{}-{}-{}".format(test_map["name_algorithm"], test_map["algorithm!idleMarks"], index),
                 bucket,
                 test_map,
                 {
                    "experimentationName": test_name,
                    "retrieveScript!getAllSinks":"true",
                    "job!jobClass":"me.florianschmidt.microbenchmark.jobs.TwoSourcesJob",                 
                    "job!rate":rate,
                    "job!rate2":5,
#                    "job!rate2":1000,
                    "job!duration": 120,                 
                    "killTaskManager!enabled": "false",
                    "killTaskManager!taskName": "map-at-level-0",
                    "killTaskManager!killDelay": 150,
                    "killTaskManager!replicaIndex": 0,
                    "killTaskManager!operatorIndex": 0,
                    "killTaskManager!gracePeriod": 0
                 },
                 no_rescale=False,
                 taskmanager_quantity=29
                )

helm delete flink --namespace default
0
helm install flink  . --set taskmanager.replicaCount=1 --set hdfs.persistence.nameNode.storageClass=local-path --set hdfs.persistence.dataNode.storageClass=local-path --set hdfs.persistence.nameNode.size=5Gi --set hdfs.persistence.dataNode.size=5Gi --set zookeeper.persistence.storageClass=local-path --set kafka.persistence.storageClass=local-path --set hdfs.enabled=true --set kafka.enabled=false --set zookeeper.enabled=false --set image.tag=1.7.2-baseline  --wait  --namespace default --timeout 180s 


{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'true', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'cp', 'experimentationName': '2021-07-09-08-28-06-mb4-v2-noprom-cp-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 10, 'job!rate2': 5, 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0}
helm delete mb4-cp-false-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n defa

{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'true', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'cp', 'experimentationName': '2021-07-09-08-56-18-mb4-v2-noprom-cp-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 100, 'job!rate2': 5, 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0}
helm delete mb4-cp-false-0 --namespace default
0
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n def

{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'true', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'cp', 'experimentationName': '2021-07-09-09-24-44-mb4-v2-noprom-cp-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 1000, 'job!rate2': 5, 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0}
helm delete mb4-cp-false-0 --namespace default
0
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n de

{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'true', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'cp', 'experimentationName': '2021-07-09-09-53-15-mb4-v2-noprom-cp-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 5000, 'job!rate2': 5, 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0}
helm delete mb4-cp-false-0 --namespace default
0
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n de

...

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA', 'experimentationName': '2021-07-09-13-20-51-mb4-v2-noprom-ka-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 10, 'job!rate2': 5, 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0}
helm delete mb4-ka-true-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA', 'experimentationName': '2021-07-09-13-49-12-mb4-v2-noprom-ka-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 100, 'job!rate2': 5, 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0}
helm delete mb4-ka-true-0 --namespace default
0
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 -

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA', 'experimentationName': '2021-07-09-14-17-57-mb4-v2-noprom-ka-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 1000, 'job!rate2': 5, 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0}
helm delete mb4-ka-true-0 --namespace default
0
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA', 'experimentationName': '2021-07-09-14-46-38-mb4-v2-noprom-ka-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 5000, 'job!rate2': 5, 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0}
helm delete mb4-ka-true-0 --namespace default
0
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA', 'experimentationName': '2021-07-09-15-15-28-mb4-v2-noprom-ka-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 10000, 'job!rate2': 5, 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0}
helm delete mb4-ka-true-0 --namespace default
0
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0

# generic mb4' (live robin intervals)

In [ ]:
bucket = "mb4b-v2-noprom"
length = 1
test_maps = [params_ar_liverobin]
#test_maps = [params_ar_leader_kafka]
for test_map in tqdm(test_maps):
    flink_params = dict(default_params_flink)
    flink_params["kafka!enabled"] = test_map["installKafka"]
    flink_params["zookeeper!enabled"] = test_map["installKafka"]
    flink_params["hdfs!enabled"] = test_map["installHdfs"]    
    install(test_map["image!tag"], flink_params)
    for length in tqdm(range(1,5)):    
        for interval in tqdm([5,10,50,100]):
            for index in tqdm(range(5)):
                test("mb1-{}-{}-{}-{}".format(test_map["name_algorithm"], length, interval, index),
                     bucket,
                     test_map,
                     {
                       "job!length":length,
                        "algorithm!liveRobinMarksInterval" : interval                     
                     },
                     no_rescale=False,
                     taskmanager_quantity=29
                    )           

helm delete flink --namespace default
0
helm install flink  . --set taskmanager.replicaCount=1 --set hdfs.persistence.nameNode.storageClass=local-path --set hdfs.persistence.dataNode.storageClass=local-path --set hdfs.persistence.nameNode.size=5Gi --set hdfs.persistence.dataNode.size=5Gi --set zookeeper.persistence.storageClass=local-path --set kafka.persistence.storageClass=local-path --set hdfs.enabled=true --set kafka.enabled=true --set zookeeper.enabled=true --set image.tag=1.7.2-ar  --wait  --namespace default --timeout 180s 


{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 5}
helm delete mb1-lr-1-5-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 17:06:21 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 17:06:22 : deplo

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 10}
helm delete mb1-lr-1-10-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 17:36:08 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 17:36:09 : dep

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 50}
helm delete mb1-lr-1-50-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 18:06:59 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 18:06:59 : dep

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 1, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 100}
helm delete mb1-lr-1-100-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 18:34:51 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 18:34:52 : d

...

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 3, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 50}
helm delete mb1-lr-3-50-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 21:50:04 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 21:50:04 : dep

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 3, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 100}
helm delete mb1-lr-3-100-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 22:17:52 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 22:17:52 : d

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 4, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 5}
helm delete mb1-lr-4-5-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 22:45:44 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 22:45:45 : deplo

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 4, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 10}
helm delete mb1-lr-4-10-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 23:13:42 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 23:13:42 : dep

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.IncreasingLengthJob', 'job!length': 4, 'job!duration': 120, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 50}
helm delete mb1-lr-4-50-0 --namespace default
1
Clean logs and old job artifacts
kubectl scale deployment -n default flink-jobmanager --replicas=0
kubectl scale deployment -n default flink-jobmanager --replicas=0
Log 0 - 2021-07-09 23:41:44 : deployment.apps/flink-jobmanager scaled
kubectl scale deployment -n default flink-taskmanager --replicas=0
kubectl scale deployment -n default flink-taskmanager --replicas=0
Log 0 - 2021-07-09 23:41:45 : dep

# obsolete : mb4'

In [ ]:
bucket = "mb4b-simple2"
length = 1
#test_maps = [params_ar_noordering, params_ar_betterbias, params_ar_leader_kafka]
#test_maps = [params_ar_leader_kafka]
test_maps = [params_ar_betterbias]
#test_maps = [params_vanilla]
flink_params = dict(default_params_flink)
flink_params["kafka!enabled"] = test_map["installKafka"]
flink_params["zookeeper!enabled"] = test_map["installKafka"]

install(test_map["image!tag"], flink_params)
for test_map in tqdm(test_maps):
    #flink_params = dict(default_params_flink)
    #flink_params["kafka!enabled"] = test_map["installKafka"]
    #flink_params["zookeeper!enabled"] = test_map["installKafka"]
    
    #install(test_map["image!tag"], flink_params)
#    for interval in tqdm([1,10,100,500,1000]):
    for interval in tqdm([1,1000]):
        for index in tqdm(range(5)):
            test_name = format("{}-{}-{}-{}".format(datetime.today().strftime('%Y-%m-%d-%H-%M-%S'), bucket, test_map["name_algorithm"], index))
            test("mb4b-{}".format(interval),
                 bucket,
                 test_map,
                 {
                    "experimentationName": test_name,
                    "retrieveScript!getAllSinks":"true",
#                    "job!jobClass":"me.florianschmidt.microbenchmark.jobs.TwoSourcesJob",                        
                    "job!mapParallelism":1,
                    "job!rate":20000,
                    "job!rate2":20000,
                    "job!duration": 120,                 
                    "algorithm!idleMarksInterval": interval,
                    "killTaskManager!enabled": "false",
                    "killTaskManager!taskName": "map-at-level-0",
                    "killTaskManager!killDelay": 150,
                    "killTaskManager!replicaIndex": 0,
                    "killTaskManager!operatorIndex": 0,
                    "killTaskManager!gracePeriod": 0
                 },
                 no_rescale=False,
                 taskmanager_quantity=3
                )

# mb4c

In [18]:
bucket = "mb4c-v2-noprom"
length = 1
test_maps = [params_ar_liverobin]

for test_map in tqdm(test_maps):
    flink_params = dict(default_params_flink)
    flink_params["kafka!enabled"] = test_map["installKafka"]
    flink_params["zookeeper!enabled"] = test_map["installKafka"]
    flink_params["hdfs!enabled"] = test_map["installHdfs"]
    
    
    rate = 500
    install(test_map["image!tag"], flink_params)
    for interval in tqdm([5,10,50,100]):
        for index in tqdm(range(5)):
            test_name = format("{}-{}-{}-{}".format(datetime.today().strftime('%Y-%m-%d-%H-%M-%S'), bucket, test_map["name_algorithm"], index))
            test("mb4c-{}".format(interval),
                 bucket,
                 test_map,
                 {
                    "experimentationName": test_name,
                    "retrieveScript!getAllSinks":"true",
                    "job!jobClass":"me.florianschmidt.microbenchmark.jobs.TwoSourcesJob",                        
                    "job!rate":500,
                    "job!rate2":rate,
                    "job!pattern1":"\"30:500\,60:250\,90:0\,120:0\,150:0\,180:250\,210:500\,240:1000\"",                    
                    "job!pattern2":"",                         
                    "job!duration": 300,                 
                    "algorithm!liveRobinMarksInterval": interval,
                    "killTaskManager!enabled": "false",
                    "killTaskManager!taskName": "map-at-level-0",
                    "killTaskManager!killDelay": 150,
                    "killTaskManager!replicaIndex": 0,
                    "killTaskManager!operatorIndex": 0,
                    "killTaskManager!gracePeriod": 0,
                    "algorithm!heartbeatEmitter!duration": 330
                 },
                 no_rescale=False,
                 taskmanager_quantity=29
                )

helm delete flink --namespace default
0
helm install flink  . --set taskmanager.replicaCount=1 --set hdfs.persistence.nameNode.storageClass=local-path --set hdfs.persistence.dataNode.storageClass=local-path --set hdfs.persistence.nameNode.size=5Gi --set hdfs.persistence.dataNode.size=5Gi --set zookeeper.persistence.storageClass=local-path --set kafka.persistence.storageClass=local-path --set hdfs.enabled=true --set kafka.enabled=true --set zookeeper.enabled=true --set image.tag=1.7.2-ar  --wait  --namespace default --timeout 180s 


{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 300, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 5, 'experimentationName': '2021-07-10-02-02-57-mb4c-v2-noprom-lr-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 500, 'job!rate2': 500, 'job!pattern1': '"30:500\\,60:250\\,90:0\\,120:0\\,150:0\\,180:250\\,210:500\\,240:1000"', 'job!pattern2': '', 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gra

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 300, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 10, 'experimentationName': '2021-07-10-02-46-38-mb4c-v2-noprom-lr-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 500, 'job!rate2': 500, 'job!pattern1': '"30:500\\,60:250\\,90:0\\,120:0\\,150:0\\,180:250\\,210:500\\,240:1000"', 'job!pattern2': '', 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gr

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 300, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 50, 'experimentationName': '2021-07-10-03-29-59-mb4c-v2-noprom-lr-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 500, 'job!rate2': 500, 'job!pattern1': '"30:500\\,60:250\\,90:0\\,120:0\\,150:0\\,180:250\\,210:500\\,240:1000"', 'job!pattern2': '', 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gr

{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.microbenchmark.jobs.TwoSourcesJob', 'job!length': 1, 'job!duration': 300, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'algorithm!type': 'LIVE_ROBIN', 'algorithm!liveRobinMarks': 'true', 'name_algorithm': 'lr', 'algorithm!heartbeatEmitter!enabled': 'true', 'algorithm!liveRobinMarksInterval': 100, 'experimentationName': '2021-07-10-04-13-28-mb4c-v2-noprom-lr-0', 'retrieveScript!getAllSinks': 'true', 'job!rate': 500, 'job!rate2': 500, 'job!pattern1': '"30:500\\,60:250\\,90:0\\,120:0\\,150:0\\,180:250\\,210:500\\,240:1000"', 'job!pattern2': '', 'killTaskManager!enabled': 'false', 'killTaskManager!taskName': 'map-at-level-0', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!g

# RW

In [ ]:
flink_params

In [ ]:
bucket = "bettercloud-v2-noprom"
length = 1
#docker_tag = "1.7.2-ar"
#test_maps = [params_vanilla, params_ar_betterbias, params_ar_leader_kafka, params_ar_leader_kafka, params_ar_leader_kafka]
test_maps = [params_vanilla, params_ar_liverobin, params_ar_leader_kafka_kill_leader, params_ar_leader_kafka_kill_follower]
#test_maps = [params_vanilla, params_ar_betterbias]

for test_map in tqdm(test_maps):
    flink_params = dict(default_params_flink)
    flink_params["kafka!enabled"] = test_map["installKafka"]
    flink_params["zookeeper!enabled"] = test_map["installKafka"]
    flink_params["hdfs!enabled"] = test_map["installHdfs"]    
    install(test_map["image!tag"], flink_params)    
    for user_rate in tqdm([100,1000,5000]):
        for index in tqdm(range(5)):    
            test_name = format("{}-{}-{}-{}".format(datetime.today().strftime('%Y-%m-%d-%H-%M-%S'), bucket, test_map["name_algorithm"], index))
            test("bc-{}-{}".format(user_rate, index),
                 bucket,
                 test_map,
                 {
                    "image!pullPolicy": "IfNotPresent",
                    "experimentationName": test_name,             
                    "workload!length": 1,
                    "workload!replicationFactor": 1,
                    "retrieveScript!getAllSinks":"true",                     
                    "job!jobClass":"me.florianschmidt.examples.bettercloud.Job",
                    "injector!enabled":"true",
                    "injector!consumerRate":user_rate,
                    "injector!controlRate":5,
                    "injector!seed": index + 1,
                    "job!duration": 300,                 
                    "killTaskManager!enabled": "true",
                    "killTaskManager!taskName": "Qualifier",
                    "killTaskManager!killDelay": 150,
                    "killTaskManager!replicaIndex": 0,
                    "killTaskManager!operatorIndex": 0,
                    "killTaskManager!gracePeriod": 0,
                    "algorithm!heartbeatEmitter!duration": 330                     
                 },
                 no_rescale=False,
                 taskmanager_quantity=29
                )

helm delete flink --namespace default
0
helm install flink  . --set taskmanager.replicaCount=1 --set hdfs.persistence.nameNode.storageClass=local-path --set hdfs.persistence.dataNode.storageClass=local-path --set hdfs.persistence.nameNode.size=5Gi --set hdfs.persistence.dataNode.size=5Gi --set zookeeper.persistence.storageClass=local-path --set kafka.persistence.storageClass=local-path --set hdfs.enabled=true --set kafka.enabled=false --set zookeeper.enabled=false --set image.tag=1.7.2-baseline  --wait  --namespace default --timeout 180s 


{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA', 'job!jobClass': 'me.florianschmidt.examples.bettercloud.Job', 'job!length': 1, 'job!duration': 300, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'true', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'cp', 'image!pullPolicy': 'IfNotPresent', 'experimentationName': '2021-07-11-17-20-16-bettercloud-v2-noprom-cp-0', 'workload!length': 1, 'workload!replicationFactor': 1, 'retrieveScript!getAllSinks': 'true', 'injector!enabled': 'true', 'injector!consumerRate': 100, 'injector!controlRate': 5, 'injector!seed': 1, 'killTaskManager!enabled': 'true', 'killTaskManager!taskName': 'Qualifier', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0, 'algorithm!heartbeatEmi

{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA', 'job!jobClass': 'me.florianschmidt.examples.bettercloud.Job', 'job!length': 1, 'job!duration': 300, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'true', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'cp', 'image!pullPolicy': 'IfNotPresent', 'experimentationName': '2021-07-11-18-02-02-bettercloud-v2-noprom-cp-0', 'workload!length': 1, 'workload!replicationFactor': 1, 'retrieveScript!getAllSinks': 'true', 'injector!enabled': 'true', 'injector!consumerRate': 1000, 'injector!controlRate': 5, 'injector!seed': 1, 'killTaskManager!enabled': 'true', 'killTaskManager!taskName': 'Qualifier', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0, 'algorithm!heartbeatEm

{'image!tag': '1.7.2-baseline', 'installKafka': 'false', 'algorithm!type': 'VANILLA', 'job!jobClass': 'me.florianschmidt.examples.bettercloud.Job', 'job!length': 1, 'job!duration': 300, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'true', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'false', 'job!sharingGroup': 'true', 'killTaskManager!waitForKill': 'True', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'cp', 'image!pullPolicy': 'IfNotPresent', 'experimentationName': '2021-07-11-18-43-49-bettercloud-v2-noprom-cp-0', 'workload!length': 1, 'workload!replicationFactor': 1, 'retrieveScript!getAllSinks': 'true', 'injector!enabled': 'true', 'injector!consumerRate': 5000, 'injector!controlRate': 5, 'injector!seed': 1, 'killTaskManager!enabled': 'true', 'killTaskManager!taskName': 'Qualifier', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0, 'algorithm!heartbeatEm

In [22]:
bucket = "bettercloud-test3"
length = 1
#docker_tag = "1.7.2-ar"
#test_maps = [params_vanilla, params_ar_betterbias, params_ar_leader_kafka, params_ar_leader_kafka, params_ar_leader_kafka]
test_maps = [params_ar_leader_kafka_kill_leader]
#test_maps = [params_vanilla, params_ar_betterbias]

for test_map in tqdm(test_maps):
    flink_params = dict(default_params_flink)
    flink_params["kafka!enabled"] = test_map["installKafka"]
    flink_params["zookeeper!enabled"] = test_map["installKafka"]
    flink_params["hdfs!enabled"] = test_map["installHdfs"]    
    install(test_map["image!tag"], flink_params)    
    for interval in [25]:
        for user_rate in tqdm([5000]):
            for index in tqdm(range(1)):    
                test_name = format("{}-{}-{}-{}".format(datetime.today().strftime('%Y-%m-%d-%H-%M-%S'), bucket, test_map["name_algorithm"], index))
                test("test{}".format(index),
                     bucket,
                     test_map,
                     {
                        "experimentationName": test_name,             
                        "workload!length": 1,
                        "workload!replicationFactor": 1,
                        "retrieveScript!getAllSinks":"true",                     
                        "job!jobClass":"me.florianschmidt.examples.bettercloud.Job",
                        "injector!enabled":"true",
                        "injector!consumerRate":user_rate,
                        "injector!controlRate":5,
                        "injector!seed": index + 1,
                        "algorithm!idleMarksInterval": interval,                     
                        "job!duration": 300,                 
                        "killTaskManager!enabled": "true",
                        "killTaskManager!taskName": "Qualifier",
                        "killTaskManager!killDelay": 150,
                        "killTaskManager!replicaIndex": 0,
                        "killTaskManager!operatorIndex": 0,
                        "killTaskManager!gracePeriod": 0             
                     },
                     no_rescale=False,
                     taskmanager_quantity=29
                    )

helm delete flink --namespace default
0
helm install flink  . --set taskmanager.replicaCount=1 --set hdfs.persistence.nameNode.storageClass=local-path --set hdfs.persistence.dataNode.storageClass=local-path --set hdfs.persistence.nameNode.size=5Gi --set hdfs.persistence.dataNode.size=5Gi --set zookeeper.persistence.storageClass=local-path --set kafka.persistence.storageClass=local-path --set hdfs.enabled=true --set kafka.enabled=true --set zookeeper.enabled=true --set image.tag=1.7.2-ar  --wait  --namespace default --timeout 180s 


{'image!tag': '1.7.2-ar', 'installKafka': 'true', 'job!jobClass': 'me.florianschmidt.examples.bettercloud.Job', 'job!length': 1, 'job!duration': 300, 'job!mapParallelism': '2', 'algorithm!checkpointing': 'false', 'algorithm!stateBackend': 'rocksdb', 'algorithm!idleMarks': 'true', 'job!sharingGroup': 'true', 'installHdfs': 'true', 'job!bufferTimeout': 5, 'name_algorithm': 'ka', 'algorithm!type': 'LEADER_KAFKA', 'killTaskManager!kazooTarget': 'leader', 'experimentationName': '2021-07-11-16-33-53-bettercloud-test3-ka-0', 'workload!length': 1, 'workload!replicationFactor': 1, 'retrieveScript!getAllSinks': 'true', 'injector!enabled': 'true', 'injector!consumerRate': 5000, 'injector!controlRate': 5, 'injector!seed': 1, 'algorithm!idleMarksInterval': 25, 'killTaskManager!enabled': 'true', 'killTaskManager!taskName': 'Qualifier', 'killTaskManager!killDelay': 150, 'killTaskManager!replicaIndex': 0, 'killTaskManager!operatorIndex': 0, 'killTaskManager!gracePeriod': 0}
helm delete test0 --namespa

In [20]:
bucket = "bettercloud-test"
length = 1
#docker_tag = "1.7.2-ar"
#test_maps = [params_vanilla, params_ar_betterbias, params_ar_leader_kafka, params_ar_leader_kafka, params_ar_leader_kafka]
test_maps = [params_vanilla, params_ar_betterbias, params_ar_leader_kafka_kill_leader, params_ar_leader_kafka_kill_follower]
#test_maps = [params_vanilla, params_ar_betterbias]

for test_map in tqdm(test_maps):
    flink_params = dict(default_params_flink)
    flink_params["kafka!enabled"] = test_map["installKafka"]
    flink_params["zookeeper!enabled"] = test_map["installKafka"]
    flink_params["hdfs!enabled"] = test_map["installHdfs"]    
    install(test_map["image!tag"], flink_params)    
    for interval in [25]:
        for user_rate in tqdm([100]):
            for index in tqdm(range(1)):    
                test_name = format("{}-{}-{}-{}".format(datetime.today().strftime('%Y-%m-%d-%H-%M-%S'), bucket, test_map["name_algorithm"], index))
                test("test{}".format(index),
                     bucket,
                     test_map,
                     {
                        "experimentationName": test_name,             
                        "workload!length": 1,
                        "workload!replicationFactor": 1,
                        "retrieveScript!getAllSinks":"true",                     
                        "job!jobClass":"me.florianschmidt.examples.bettercloud.Job",
                        "injector!enabled":"true",
                        "injector!consumerRate":user_rate,
                        "injector!controlRate":5,
                        "injector!seed": index + 1,
                        "algorithm!idleMarksInterval": interval,                     
                        "job!duration": 300,                 
                        "killTaskManager!enabled": "true",
                        "killTaskManager!taskName": "Qualifier",
                        "killTaskManager!killDelay": 150,
                        "killTaskManager!replicaIndex": 0,
                        "killTaskManager!operatorIndex": 0,
                        "killTaskManager!gracePeriod": 0             
                     },
                     no_rescale=False,
                     taskmanager_quantity=29
                    )

helm delete flink --namespace default
0



KeyboardInterrupt: 